In [ ]:
import numpy as np
import stim
from qiskit import QuantumCircuit

from mqt.qecc.codes import construct_quasi_cyclic_code
from mqt.qecc.se_synthesis import OptimalSyndromeExtractionEncoder

In [ ]:
# Check matrices for the Steane Code
steane_x_checks = np.array([[1, 1, 0, 0, 1, 1, 0], [1, 0, 1, 0, 1, 0, 1], [0, 0, 0, 1, 1, 1, 1]])
steane_z_checks = steane_x_checks.copy()

x_checks = np.array([
    [0, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 0, 1, 0],
])
z_checks = data = np.array([
    [0, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 0, 1, 0],
])

# Decoding Circuit for the Steane Code

In [ ]:
encoder = OptimalSyndromeExtractionEncoder((x_checks, z_checks), 4)
res = encoder.solve()
encoder.get_circuit().draw(output="mpl")

# Decoder Circuits for Quasi-Cyclic Codes

In [ ]:
# Near Depth-optimal circuit
# This should be fast
code = construct_quasi_cyclic_code(72)
encoder = OptimalSyndromeExtractionEncoder(code, 8)

res = encoder.solve()
encoder.get_circuit().depth()

In [ ]:
# ODepth-optimal Circuit
# WARNING: This will take a while
code = construct_quasi_cyclic_code(72)
encoder = OptimalSyndromeExtractionEncoder(code, 7)

res = encoder.solve()
encoder.get_circuit().depth()

In [ ]:
def to_stim(qiskit_circ: QuantumCircuit, initialized_qubits="all", qubit_offset=0):
    stim_circ = stim.Circuit()
    if initialized_qubits is not None:
        if initialized_qubits == "all":
            initialized_qubits = list(range(qiskit_circ.num_qubits))
        stim_circ.append("R", initialized_qubits)

    for instr in qiskit_circ:
        gate = instr.operation.name
        if gate == "measure":
            gate = "MR"
        if gate == "barrier":
            continue
        qubits = [qiskit_circ.find_bit(bit).index + qubit_offset for bit in instr.qubits]
        stim_circ.append(gate, qubits)

    return stim_circ

In [ ]:
to_stim(encoder.get_circuit())

In [ ]:
[np.where(x)[0] for x in x_checks]